In [1]:
import os

openai_api_key = os.getenv("OPENAI_API_KEY")

In [2]:
from langchain_core.runnables import RunnableMap, RunnablePassthrough, RunnableLambda
from langchain_core.prompts import format_document
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts.chat import ChatPromptTemplate
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.prompts.prompt import PromptTemplate
from operator import itemgetter
from langchain_community.vectorstores.faiss import FAISS
from langchain_openai import OpenAIEmbeddings

In [5]:
vectorstore = FAISS.from_texts(
    [
        "The weather in Paris today is sunny with a high of 18°C and a low of 10°C.",
        "In New York, the temperature is around 5°C with cloudy skies and occasional rain showers.",
        "Tokyo is experiencing mild weather with temperatures ranging from 12°C to 16°C, with light winds.",
    ],
    embedding=OpenAIEmbeddings(),
)
retriever = vectorstore.as_retriever()

In [4]:
_template = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)

In [8]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
ANSWER_PROMPT = ChatPromptTemplate.from_template(template)

In [ ]:
DEFAULT_DOCUMENT_PROMPT = PromptTemplate.from_template(template="{page_content}")

def _combine_documents(
    docs, document_prompt=DEFAULT_DOCUMENT_PROMPT, document_separator="\n\n"
):
    doc_strings = [format_document(doc, document_prompt) for doc in docs] #format the documents with the document prompt
    return document_separator.join(doc_strings) #join the documents with the separator

In [7]:
from typing import List, Union
from langchain.schema import HumanMessage, SystemMessage, AIMessage

#list with the chat history
def _format_chat_history(chat_history: List[Union[HumanMessage, SystemMessage, AIMessage]]) -> str: 
    buffer = ""
    for dialogue_turn in chat_history:
        if isinstance(dialogue_turn, HumanMessage):
            buffer += "\nHuman: " + dialogue_turn.content
        elif isinstance(dialogue_turn, AIMessage):
            buffer += "\nAssistant: " + dialogue_turn.content
        elif isinstance(dialogue_turn, SystemMessage):
            buffer += "\nSystem: " + dialogue_turn.content
    return buffer

In [10]:

_inputs = RunnableMap( #combines different parts into an execution chain.
    standalone_question=RunnablePassthrough.assign( #key-value pair
        chat_history=lambda x: _format_chat_history(x["chat_history"])
    )
    | CONDENSE_QUESTION_PROMPT
    | ChatOpenAI(model="gpt-4o-mini", temperature=0)
    | StrOutputParser(),
)
_context = {
    "context": itemgetter("standalone_question") | retriever | _combine_documents, 
    #After extracting the standalone_question under the "context" key,
    # the retriever is put into a search operation, then _combine_documents combines the texts
    "question": lambda x: x["standalone_question"],
}
conversational_qa_chain = _inputs | _context | ANSWER_PROMPT | ChatOpenAI(model="gpt-4o-mini")

In [14]:
result = conversational_qa_chain.invoke(
    {
        "question": "what is the weather in Paris?",
        "chat_history": [],
    }
)
print(result.content)

The weather in Paris today is sunny with a high of 18°C and a low of 10°C.


In [ ]:
from operator import itemgetter
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(
    return_messages=True, output_key="answer", input_key="question"
)

In [17]:
# First we add a step to load memory
# This adds a "memory" key to the input object
loaded_memory = RunnablePassthrough.assign(
    chat_history=RunnableLambda(memory.load_memory_variables) | itemgetter("history"),
)
# Now we calculate the standalone question
standalone_question = {
    "standalone_question": {
        "question": lambda x: x["question"],
        "chat_history": lambda x: _format_chat_history(x["chat_history"]),
    }
    | CONDENSE_QUESTION_PROMPT
    | ChatOpenAI(model="gpt-4o-mini", temperature=0)
    | StrOutputParser(),
}
# Now we retrieve the documents

# This is REALLY IMPORTANT as the chain above becomes StrOutputParser() so it will only have one key, which gets passed to the retriever!
retrieved_documents = {
    "docs": itemgetter("standalone_question") | retriever,
    "question": lambda x: x["standalone_question"],
}
# Now we construct the inputs for the final prompt
final_inputs = {
    "context": lambda x: _combine_documents(x["docs"]),
    "question": itemgetter("question"),
}
# And finally, we do the part that returns the answers
answer = {
    "answer": final_inputs | ANSWER_PROMPT | ChatOpenAI(),
    "docs": itemgetter("docs"),
}
# And now we put it all together!
final_chain = loaded_memory | standalone_question | retrieved_documents | answer

In [27]:
inputs = {"question": "what is the situation in Tokyo?"}
result = final_chain.invoke(inputs)
print(result)

{'answer': AIMessage(content='Tokyo is experiencing mild weather with temperatures ranging from 12°C to 16°C, with light winds.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 89, 'total_tokens': 112, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-64884a1a-2777-48b0-9cab-aea250addd92-0', usage_metadata={'input_tokens': 89, 'output_tokens': 23, 'total_tokens': 112, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}), 'docs': [Document(id='77ca0814-24bd-4e13-9cf0-359df70ae403', metadata={}, page_content='Tokyo is experiencing mild weather with temperatures ranging from 12°C to 16°C, with light w

In [28]:
# Note that the memory does not save automatically
# This will be improved in the future
# For now you need to save it yourself
memory.save_context(inputs, {"answer": result["answer"].content})

In [29]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='what is the weather in New York?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='In New York, the weather is around 5°C with cloudy skies and occasional rain showers.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='what is the situation in Tokyo?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Tokyo is experiencing mild weather with temperatures ranging from 12°C to 16°C, with light winds.', additional_kwargs={}, response_metadata={})]}